In [9]:
!pip install yfinance --upgrade

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/73/b5/d50eec88bc731bb8570ae42a9b764a36144e217361c33fa068391ff59ba3/yfinance-0.2.61-py2.py3-none-any.whl.metadata
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for frozendict>=2.3.4 from https://files.pythonhosted.org/packages/04/13/d9839089b900fa7b479cce495d62110cddc4bd5630a04d8469916c0e79c5/frozendict-2.4.6-py311-none-any.whl.metadata
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
      --------------------------------------- 0.1/3.0 MB 975.2 kB/s eta 0:00:04
     ---- ----------------------------------- 0.4/3.0 MB 3.3 MB/s eta 0:00:01
     -------- ------------------------------- 0.6/3.0 MB 3.6 MB

In [2]:
import requests, pandas as pd

key = "I8SFBCEFPANO2KP0"
sym = "AAPL"
url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={sym}&apikey={key}"
data = requests.get(url).json()
print(data["MarketCapitalization"], data["DebtToEquity"])


KeyError: 'DebtToEquity'

# Debt to Equity (Alpha)

In [4]:
import requests, json

key = "I8SFBCEFPANO2KP0"
sym = "AAPL"

# market-cap
ov = requests.get(
    f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={sym}&apikey={key}"
).json()
print("Mkt-cap:", ov.get("MarketCapitalization"))

# debt-to-equity
bs = requests.get(
    f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={sym}&apikey={key}"
).json()
q = bs["quarterlyReports"][0]          # latest quarter
total_debt = float(q["shortTermDebt"]) + float(q["longTermDebt"])
equity = float(q["totalShareholderEquity"])
print("D/E:", round(total_debt / equity, 2))
#``` :contentReference[oaicite:0]{index=0}
#::contentReference[oaicite:1]{index=1}
#

Mkt-cap: 2916513743000
D/E: 1.47


# Debt to Assets ratio (Alpha)

In [5]:
import requests

def debt_to_assets(symbol: str, api_key: str) -> float:
    """
    Returns latest quarterly Debt-to-Assets ratio:
        (short-term + long-term interest-bearing debt) / total assets
    """
    url = (
        f"https://www.alphavantage.co/query?"
        f"function=BALANCE_SHEET&symbol={symbol}&apikey={api_key}"
    )
    js = requests.get(url, timeout=10).json()
    q = js["quarterlyReports"][0]          # most-recent quarter

    debt = float(q.get("shortTermDebt", 0) or 0) \
         + float(q.get("longTermDebt", 0) or 0)
    assets = float(q["totalAssets"])       # present in feed :contentReference[oaicite:0]{index=0}

    return round(debt / assets, 4)

# --- demo -------------------------------------------------
key = "I8SFBCEFPANO2KP0"
print("AAPL debt/assets:", debt_to_assets("AAPL", key))


AAPL debt/assets: 0.2964


# Yahoo Finance

In [12]:
bs = tkr.quarterly_balance_sheet.T

# Calculate debt using available fields
debt = bs.get("Long Term Debt", 0).fillna(0) + bs.get("Current Debt", 0).fillna(0)
assets = bs.get("Total Assets", 1).fillna(1)  # fallback to 1 to avoid zero-div error

dta = (debt / assets).rename("debt_to_assets")
print(dta.head())


2025-03-31    0.296426
2024-12-31    0.281323
2024-09-30    0.292150
2024-06-30    0.305490
2024-03-31    0.309978
Name: debt_to_assets, dtype: float64


In [13]:
pip install requests pandas yfinance tqdm

Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests, pandas as pd, yfinance as yf, time
from tqdm import tqdm

HEADERS = {"User-Agent": "Mozilla/5.0"}
CIK_URL  = "https://www.sec.gov/files/company_tickers_exchange.json"
FACT_URL = "https://data.sec.gov/api/xbrl/companyfacts/CIK{:010d}.json"

# Step 1: Build ticker → CIK map
tick_map = {v["ticker"]: int(k) for k, v in requests.get(CIK_URL, headers=HEADERS).json().items()}

# Use a small list to test first
tickers = ["AAPL", "MSFT", "AMZN"]

out_rows = []
for tk in tqdm(tickers):
    cik = tick_map.get(tk)
    if not cik:
        continue

    # Step 2: Get XBRL Company Facts with error handling
    try:
        resp = requests.get(FACT_URL.format(cik), headers=HEADERS, timeout=10)
        if resp.status_code != 200 or not resp.text.startswith("{"):
            raise ValueError(f"{tk}: bad SEC response")
        facts = resp.json()["facts"]["us-gaap"]
    except Exception as e:
        print(f"{tk}: skip –", e)
        time.sleep(1)
        continue

    # Step 3: Extract quarterly debt and assets
    def extract_series(tags):
        for tag in tags:
            if tag in facts and "USD" in facts[tag]["units"]:
                return {
                    v.get("end", v.get("fp")): float(v["val"])
                    for v in facts[tag]["units"]["USD"]
                    if v.get("fp", "").startswith("Q")
                }
        return {}

    debt = extract_series(["Debt", "LongTermDebt", "LongTermDebtNoncurrent"])
    assets = extract_series(["Assets"])
    if not debt or not assets:
        continue

    # Step 4: Get market cap from yfinance
    try:
        yf_tkr = yf.Ticker(tk)
        prices = yf_tkr.history(period="20y", interval="1d")["Close"].resample("Q").last()
        shares = yf_tkr.get_shares_full().set_index("Date")["Shares"].resample("Q").last()
        mcap = (prices * shares).dropna()
        mcap.index = mcap.index.strftime("%Y-%m-%d")
    except Exception as e:
        print(f"{tk} yfinance error –", e)
        continue

    # Step 5: Merge by matching quarter-end
    common_dates = set(debt) & set(assets) & set(mcap.index)
    for q in common_dates:
        try:
            dta = debt[q] / assets[q]
            out_rows.append([tk, q, round(dta, 4), int(mcap[q])])
        except:
            continue

    time.sleep(1)  # be nice to SEC’s API

# Step 6: Output results
df = pd.DataFrame(out_rows, columns=["ticker", "quarter_end", "debt_to_assets", "market_cap"])
df = df.sort_values(["ticker", "quarter_end"])
print(df.head())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# FMP

In [25]:
import requests, pandas as pd

API  = "7cNMpVzb43GKtm05iRTDWJtyJXSylX8J"
TICK = "AAPL"

def get_json(url, params):
    js = requests.get(url, params=params, timeout=10).json()
    if not isinstance(js, list):
        raise RuntimeError(js)
    return js

# -------- balance-sheet (quarterly) --------
bs = get_json(
    f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{TICK}",
    {"period": "quarter", "limit": 40, "apikey": API},
)

bs_df = (
    pd.DataFrame(bs)
      .loc[:, ["date", "shortTermDebt", "longTermDebt", "totalAssets"]]
      .assign(date=lambda d: pd.to_datetime(d.date),
              quarter=lambda d: d.date.dt.to_period("Q"),
              debt_to_assets=lambda d:
                  (d.shortTermDebt.fillna(0)+d.longTermDebt.fillna(0)) / d.totalAssets)
)

# -------- market-cap (pick last trading day each quarter) --------
mc = get_json(
    f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{TICK}",
    {"limit": 10000, "apikey": API},
)

mc_qtr = (
    pd.DataFrame(mc)
      .assign(date=lambda d: pd.to_datetime(d.date),
              quarter=lambda d: d.date.dt.to_period("Q"))
      .sort_values("date")
      .drop_duplicates("quarter", keep="last")           # one row per quarter
      .rename(columns={"marketCap": "mkt_cap"})
      [["quarter", "mkt_cap"]]
)

# -------- merge & view --------
out = (
    bs_df.merge(mc_qtr, on="quarter", how="left")
         .sort_values("date", ascending=False)
         .reset_index(drop=True)
)

print(out.head())


        date  shortTermDebt  longTermDebt   totalAssets quarter  \
0 2025-03-29    19620000000   78566000000  331233000000  2025Q1   
1 2024-12-28    12843000000   83956000000  344085000000  2024Q4   
2 2024-09-28    22511000000   96548000000  364980000000  2024Q3   
3 2024-06-29    15108000000   86196000000  331612000000  2024Q2   
4 2024-03-30    12759000000   91831000000  337411000000  2024Q1   

   debt_to_assets       mkt_cap  
0        0.296426  3.330635e+12  
1        0.281323  3.754818e+12  
2        0.326207  3.514042e+12  
3        0.305490  3.219858e+12  
4        0.309978  2.641796e+12  


In [29]:
import requests, pandas as pd, time

API = "7cNMpVzb43GKtm05iRTDWJtyJXSylX8J"

def get_json(url, params={}):
    params["apikey"] = API
    js = requests.get(url, params=params, timeout=10).json()
    if not isinstance(js, list):
        return None
    return js

# Get all US tickers
tickers = get_json("https://financialmodelingprep.com/api/v3/stock/list")
tickers = [d["symbol"] for d in tickers if d["exchangeShortName"] in ["NYSE", "NASDAQ"]]

# Container for all rows
all_data = []

for ticker in tickers[:100]:  # limit to 100 for testing
    try:
        bs = get_json(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}", {"period": "quarter", "limit": 20})
        mc = get_json(f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{ticker}", {"limit": 1000})
        px = get_json(f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}", {"serietype": "line", "timeseries": 1000})
        profile = get_json(f"https://financialmodelingprep.com/api/v3/profile/{ticker}")
        if not all([bs, mc, px, profile]): continue

        industry = profile[0].get("industry")
        sector = profile[0].get("sector")

        bs_df = (
            pd.DataFrame(bs)
            .loc[:, ["date", "shortTermDebt", "longTermDebt", "totalAssets"]]
            .assign(date=lambda d: pd.to_datetime(d.date),
                    quarter=lambda d: d.date.dt.to_period("Q"),
                    debt_to_assets=lambda d:
                        (d.shortTermDebt.fillna(0) + d.longTermDebt.fillna(0)) / d.totalAssets)
        )

        mc_df = (
            pd.DataFrame(mc)
            .assign(date=lambda d: pd.to_datetime(d.date),
                    quarter=lambda d: d.date.dt.to_period("Q"))
            .sort_values("date")
            .drop_duplicates("quarter", keep="last")
            .rename(columns={"marketCap": "mkt_cap"})
            [["quarter", "mkt_cap"]]
        )

        px_df = (
            pd.DataFrame(px["historical"])
            .assign(date=lambda d: pd.to_datetime(d.date),
                    quarter=lambda d: d.date.dt.to_period("Q"))
            .sort_values("date")
            .drop_duplicates("quarter", keep="last")
            .rename(columns={"close": "stock_price"})
            [["quarter", "stock_price"]]
        )

        merged = (
            bs_df.merge(mc_df, on="quarter", how="left")
                 .merge(px_df, on="quarter", how="left")
                 .assign(ticker=ticker, industry=industry, sector=sector)
                 [["quarter", "ticker", "industry", "sector", "debt_to_assets", "mkt_cap", "stock_price"]]
        )

        all_data.append(merged)

        time.sleep(1)  # to avoid hitting rate limits
    except Exception as e:
        print(f"Checking {ticker}...")
        if not all([bs, mc, px, profile]):
            print(f"Missing data for {ticker}, skipping.")
            continue
# Final dataset
final_df = pd.concat(all_data).sort_values(["ticker", "quarter"])
final_df.reset_index(drop=True, inplace=True)
print(final_df.head())


ValueError: No objects to concatenate